In [1]:
import numpy as np
import tensorflow as tf
import collections
import pandas as pd

In [2]:
train_df=pd.read_pickle('./data/train_df_nopunc.pickle')


In [3]:
a=train_df['text'].tolist()
txt=' '.join(a)
del a

In [4]:
txt=txt.lower()

In [5]:
n_words=1000000
words=txt.split()
count=collections.Counter(words).most_common(n_words - 1)
count.insert(0,('rare_word',-1))
i=0
word_dict=dict()
for word,num in count:
    word_dict[word]=i
    i+=1

words_num_list=list()
rare_word_count=0
for word in words:
    ind=word_dict.get(word,0)
    words_num_list.append(ind)
    if ind==0:
        rare_word_count+=1
count[0]=('rare_word',rare_word_count)

In [6]:
import random
global_counter=0
def generate_batch_seq(batch_size,skip_window,num_skip):
    global global_counter
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    training_labels = np.ndarray(shape=(batch_size,1), dtype=np.int32)    
    window_span=skip_window * 2 + 1
    for i in range(batch_size//num_skip):
        if global_counter+window_span>len(words_num_list):
            global_counter=0
        word_window=words_num_list[global_counter:global_counter+window_span]
        context_indices_in_window=[w for w in range(window_span) if w!=skip_window]
        selected_indices=random.sample(context_indices_in_window,num_skip)
        for j,selected_index in enumerate(selected_indices):
            context_word=word_window[selected_index]
            center_word=word_window[skip_window]
            batch[i*num_skip+j]=context_word
            training_labels[i*num_skip+j]=center_word
        global_counter +=1
    return batch,training_labels

In [7]:
del train_df

In [8]:
batch_size = 128
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 2  # How many words to consider left and right.
num_skip = 2  # How many times to reuse an input to generate a label.
num_sampled = 64  # Number of negative examples to sample.
restore_sess=0

graph=tf.Graph()
with graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[batch_size])
    labels = tf.placeholder(tf.int32, shape=[batch_size,1])
    embeddings = tf.Variable(tf.random_uniform([n_words, embedding_size], -1.0, 1.0))
    #embeddings = tf.Variable(final_embeddings)
    embed = tf.nn.embedding_lookup(embeddings, inputs)
    nce_loss_weights = tf.Variable(tf.truncated_normal([n_words, embedding_size],stddev=1.0 / np.sqrt(embedding_size)))
    nce_loss_biases = tf.Variable(tf.zeros([n_words]))
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_loss_weights,biases=nce_loss_biases,labels=labels,inputs=embed,
                                         num_sampled=num_sampled,num_classes=n_words))
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    #optimizer = tf.train.AdamOptimizer().minimize(loss)
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    init = tf.global_variables_initializer()
    saver=tf.train.Saver()

steps=1000000
config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
with tf.Session(graph=graph,config=config) as session:
    if restore_sess==1:
        saver.restore(session,'./ckpt/skipgram_amazon.ckpt')
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(steps):
        batch_inputs, batch_labels = generate_batch_seq(batch_size, skip_window,num_skip)
        feed_dict = {inputs: batch_inputs, labels: batch_labels}
        _, loss_val = session.run(
        [optimizer, loss],
        feed_dict=feed_dict)
        if step%1000==0:
            print(step,'loss=',loss_val)
            
    saver.save(session,'./ckpt/skipgram_amazon.ckpt')
    final_embeddings=normalized_embeddings.eval()

Initialized
0 loss= 362.456
1000 loss= 255.112
2000 loss= 210.831
3000 loss= 209.682
4000 loss= 191.136
5000 loss= 153.3
6000 loss= 204.06
7000 loss= 134.71
8000 loss= 178.065
9000 loss= 158.172
10000 loss= 201.492
11000 loss= 152.143
12000 loss= 105.094
13000 loss= 88.0355
14000 loss= 123.605
15000 loss= 139.512
16000 loss= 69.6525
17000 loss= 130.037
18000 loss= 137.007
19000 loss= 92.6008
20000 loss= 80.3961
21000 loss= 118.776
22000 loss= 129.41
23000 loss= 174.568
24000 loss= 125.567
25000 loss= 103.651
26000 loss= 91.5719
27000 loss= 55.8084
28000 loss= 103.119
29000 loss= 94.2873
30000 loss= 132.64
31000 loss= 91.2267
32000 loss= 141.809
33000 loss= 94.4632
34000 loss= 98.974
35000 loss= 57.2071
36000 loss= 96.4389
37000 loss= 47.6846
38000 loss= 69.5843
39000 loss= 149.928
40000 loss= 120.113
41000 loss= 51.5818
42000 loss= 92.4053
43000 loss= 44.8467
44000 loss= 39.4408
45000 loss= 78.0928
46000 loss= 40.635
47000 loss= 92.8686
48000 loss= 70.8508
49000 loss= 26.8771
50000 los

398000 loss= 4.75186
399000 loss= 8.13961
400000 loss= 12.861
401000 loss= 4.59393
402000 loss= 4.6815
403000 loss= 16.1662
404000 loss= 4.47623
405000 loss= 31.8226
406000 loss= 17.2117
407000 loss= 4.66666
408000 loss= 5.86273
409000 loss= 4.27969
410000 loss= 6.2808
411000 loss= 4.28573
412000 loss= 6.8181
413000 loss= 7.02869
414000 loss= 4.66766
415000 loss= 4.87405
416000 loss= 5.16655
417000 loss= 20.5234
418000 loss= 10.3612
419000 loss= 9.38875
420000 loss= 29.5113
421000 loss= 4.0614
422000 loss= 4.78751
423000 loss= 20.2849
424000 loss= 4.61323
425000 loss= 3.53558
426000 loss= 11.2309
427000 loss= 5.18555
428000 loss= 4.75514
429000 loss= 4.75249
430000 loss= 5.00557
431000 loss= 28.9355
432000 loss= 29.9887
433000 loss= 4.67735
434000 loss= 4.65641
435000 loss= 4.57053
436000 loss= 11.0733
437000 loss= 4.17341
438000 loss= 5.33555
439000 loss= 4.85981
440000 loss= 5.17513
441000 loss= 6.54409
442000 loss= 5.2277
443000 loss= 3.98153
444000 loss= 12.3572
445000 loss= 4.1443

791000 loss= 7.48354
792000 loss= 4.6634
793000 loss= 19.0877
794000 loss= 4.87795
795000 loss= 5.17018
796000 loss= 16.9023
797000 loss= 5.21196
798000 loss= 5.65979
799000 loss= 14.2727
800000 loss= 4.86168
801000 loss= 4.47149
802000 loss= 4.95987
803000 loss= 4.92313
804000 loss= 7.88829
805000 loss= 5.18504
806000 loss= 10.3493
807000 loss= 4.60149
808000 loss= 4.83744
809000 loss= 4.62306
810000 loss= 4.48364
811000 loss= 4.74476
812000 loss= 17.5494
813000 loss= 4.5536
814000 loss= 4.73604
815000 loss= 4.66203
816000 loss= 8.78923
817000 loss= 16.599
818000 loss= 10.1033
819000 loss= 7.77062
820000 loss= 8.89803
821000 loss= 4.23283
822000 loss= 7.60519
823000 loss= 4.35733
824000 loss= 6.59965
825000 loss= 4.656
826000 loss= 4.21586
827000 loss= 5.13169
828000 loss= 4.40661
829000 loss= 8.34948
830000 loss= 4.74023
831000 loss= 4.8398
832000 loss= 4.32539
833000 loss= 4.43347
834000 loss= 4.19985
835000 loss= 4.48017
836000 loss= 4.37557
837000 loss= 4.44863
838000 loss= 4.8451

In [9]:
a=final_embeddings[word_dict['seven']]
a

array([  1.84558760e-02,  -1.65749788e-02,   6.94774389e-02,
         1.66705176e-02,   9.36170965e-02,   2.47048289e-02,
        -9.49273482e-02,  -1.10419951e-02,  -6.75776675e-02,
        -6.54449016e-02,  -1.13923033e-03,  -7.25404024e-02,
         1.13724701e-01,  -4.87168394e-02,  -7.93844610e-02,
         4.18764576e-02,   4.04779948e-02,   2.11862754e-02,
         1.00889988e-01,  -6.11836463e-02,  -3.54630910e-02,
         2.43001012e-03,   7.64218019e-03,  -7.71658495e-02,
        -5.49962651e-03,  -4.72761840e-02,  -4.50324789e-02,
        -2.82826531e-03,  -7.33576789e-02,   1.08757764e-01,
        -7.99919963e-02,  -8.49682465e-02,   2.48433631e-02,
        -4.57409509e-02,  -5.89199597e-03,  -4.47022244e-02,
        -1.14896826e-01,  -1.05048895e-01,  -1.65068191e-02,
        -2.51138788e-02,  -5.13973907e-02,  -4.16491590e-02,
         4.94319014e-02,   4.05028649e-02,   4.66745943e-02,
        -7.11736754e-02,  -5.20588011e-02,  -2.15571541e-02,
         3.22021618e-02,

In [10]:
reverse_dict={v: k for k, v in word_dict.items()}

In [12]:
# pylint: disable=g-import-not-at-top
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

tsne = TSNE(
  perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
plot_only = 500
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [reverse_dict[i] for i in range(plot_only)]

In [ ]:
import os
def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')

    plt.savefig(filename)
plot_with_labels(low_dim_embs, labels, 'tsne.png')

In [11]:
np.savez('./data/word2vec_1M_amazon.npz',final_embeddings,word_dict,reverse_dict)